In [1]:
from aestheval.data.datasets import AVADataset
import torch
from tqdm import tqdm
SPLITS = ('train', 'validation', 'test')

ava_dataset = [AVADataset(split, comments_path= "../data/ava",) for split in SPLITS]

In [2]:
dataset = torch.utils.data.ConcatDataset(ava_dataset)

In [3]:
gt_scores, pred_scores = zip(*[(data['im_score'], data['mean_score']) for _, data in tqdm(dataset)])



100%|██████████| 253961/253961 [00:00<00:00, 760472.22it/s]


In [4]:
from scipy import stats
import sklearn.metrics as sm

srcc = stats.spearmanr(gt_scores, pred_scores)[0]
print("SRCC =", srcc)
mse = round(sm.mean_squared_error(gt_scores, [score*10 for score in pred_scores]), 4)
print("MSE =", mse)
lcc = stats.pearsonr(gt_scores, pred_scores)[0]
print("LCC =", lcc)

SRCC = 0.6418159938404643
MSE = 5.9397
LCC = 0.6424464694494703
